In [2]:
import wandb
import ultralytics
from ultralytics import YOLO
import torch
import cv2
import pandas as pd
import os

#check yolo version and using cuda
# print(ultralytics.__version__, torch.cuda.is_available())

In [5]:
!nvidia-smi

Thu Feb  6 21:58:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   41C    P8             13W /  125W |    1409MiB /   6144MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
wandb.login()

True

Loading the model yoloV8 

In [8]:
#training model
model = YOLO('yolov8x.pt')

100%|██████████| 131M/131M [00:15<00:00, 8.76MB/s] 
